In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
import pickle

In [19]:
data = pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [20]:
# preprocessing the data
data = data.drop(['RowNumber','CustomerId','Surname'],axis=1)

In [21]:
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

In [22]:
# One hot encoding 'Geography'
onehot_encoder_geo = OneHotEncoder(handle_unknown='ignore',sparse_output=False)
geo_encoded = onehot_encoder_geo.fit_transform(data[['Geography']])
geo_encoded_df = pd.DataFrame(geo_encoded,columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [23]:
# combining encoded columns with the original data
data = pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [24]:
# splitting out data into independent and target features
x = data.drop('EstimatedSalary',axis=1)
y = data['EstimatedSalary']

In [25]:
# spilitting the data in training and testing sets
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [26]:
# scaling these features
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [28]:
# saving the encoders and scaler for later use
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('onehot_encoder_geo.pkl','wb') as file:
    pickle.dump(onehot_encoder_geo,file)

with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

### ANN Regression Problem Statement


In [29]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [30]:
# Building our model
model = Sequential([
    Dense(64, activation='relu',input_shape=(x_train.shape[1],)),
    Dense(32,activation='relu'),
    Dense(1)
])

# compiling the model
model.compile(optimizer='adam',loss='mean_absolute_error',metrics=['mae'])

model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [31]:
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

# setting up tensorboard
log_dir = "regressionlogs/fit/" + datetime.datetime.now().strftime("Y%m%d-%H%M%S")
tensorBoard_callback = TensorBoard(log_dir = log_dir , histogram_freq = 1)

In [32]:
# setting up early stopping
early_Stopping_callback = EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True)

In [33]:
# Training the model
history = model.fit(
    x_train, y_train,
    validation_data = (x_test,y_test),
    epochs = 100,
    callbacks = [early_Stopping_callback , tensorBoard_callback]
)

Epoch 1/100


250/250 [==============================] - 5s 10ms/step - loss: 100369.0391 - mae: 100369.0391 - val_loss: 98477.4375 - val_mae: 98477.4375
Epoch 2/100
250/250 [==============================] - 2s 6ms/step - loss: 99471.1172 - mae: 99471.1172 - val_loss: 96653.8984 - val_mae: 96653.8984
Epoch 3/100
250/250 [==============================] - 2s 9ms/step - loss: 96328.2500 - mae: 96328.2500 - val_loss: 92029.3594 - val_mae: 92029.3594
Epoch 4/100
250/250 [==============================] - 2s 8ms/step - loss: 90237.3672 - mae: 90237.3672 - val_loss: 84530.1094 - val_mae: 84530.1094
Epoch 5/100
250/250 [==============================] - 2s 7ms/step - loss: 81597.6641 - mae: 81597.6641 - val_loss: 75126.7656 - val_mae: 75126.7656
Epoch 6/100
250/250 [==============================] - 2s 7ms/step - loss: 71886.1406 - mae: 71886.1406 - val_loss: 66046.8594 - val_mae: 66046.8594
Epoch 7/100
250/250 [==============================] - 2s 9ms/step - loss: 63157.0352 - mae: 63157.03

In [34]:
%load_ext tensorboard

In [35]:
%tensorboard --logdir regression_logs/fit

In [36]:
# evaluating the model on test data
test_loss , test_mae = model.evaluate(x_test , y_test)
print(f'Test MAE : {test_mae}')

63/63 [==============================] - 0s 3ms/step - loss: 50230.4922 - mae: 50230.4922
Test MAE : 50230.4921875


In [37]:
model.save('regression_model.h5')

c:\Users\shesh\anaconda3\envs\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
